In [1]:
import pandas as pd
import requests
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

c:\Users\Admin\miniconda3\envs\recsys\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# Load the DataFrame
df = pd.read_csv('/kaggle/input/recsys-listings/data.csv')

# Initialize Blip processor and model

# Function to generate caption for an image
def generate_caption(image_path):
    # Load image
    raw_image = Image.open(image_path).convert('RGB')

    # Perform conditional image captioning
    text = "a photograph of"
    inputs = processor(raw_image, text, return_tensors="pt").to("cuda", torch.float16)
    out = model.generate(**inputs, min_length=20)  # Adjust min_length as needed
    caption = processor.decode(out[0], skip_special_tokens=True)

    return caption

# Function to process each row
def process_row(row):
    # Extract URLs from 'originalPhotos' column
    photo_urls = eval(row['originalPhotos'])  # Convert string representation to list

    # Initialize a list to store captions
    captions = []

    # Loop through each photo URL
    for url in photo_urls:
        # Send request to download the image
        response = requests.get(url)
        image_path = url.split('/')[-1]  # Extract image filename
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Generate caption for the image
        caption = generate_caption(image_path)
        captions.append(caption)

    # Return concatenated captions
    return '; '.join(captions)

# Apply the function to each row and add the result to a new column



In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")


In [ ]:
df['image_captions'] = df.apply(process_row, axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_data.csv', index=False)

print("Image captions generated and added to the dataset.")